# Challenge Amadeus
## 1. 
Count lines
At first we hace to open the file bookings.csv.bz2

In [237]:
import pandas as pd
import numpy as np
import matplotlib as plot
import bz2

My file is in /home/dsc/Data/challenge/bookings.csv.bz2

In [238]:
pFolder = '/home/dsc/Data/challenge/'
fBookings = pFolder+'bookings.csv.bz2'
fSearches = pFolder+'searches.csv.bz2'

I try open the file with read_csv and create a DataFrame for the other exercices

In [239]:
#dataBookings = pd.read_csv(fBookings,delimiter='^',iterator=True,chunksize=100000)


In [240]:
#count=0
#for data in dataBookings:
#    count +=1
#print(count)

In [241]:
#dfBookings = pd.concat(dataBookings)

In [226]:
count = 0
with bz2.BZ2File(fBookings, "r") as bz:
    for line in bz:
        count += 1
print(count)

10000011


In [227]:
count = 0
with bz2.BZ2File(fSearches, "r") as bz:
    for line in bz:
        count += 1
print(count)

20390199


## 2. Exercise 2
Top 10 arrival airports in the world in 2013 of bookings file

We will create a sample of bookings and we can play confortable

In [ ]:
! bzcat {pFolder}bookings.csv.bz2 | head -100000 | bzip2 -c > {pFolder}bookings.sample.csv.bz2 

In [ ]:
! bzcat {pFolder}bookings.sample.csv.bz2 | wc 

In [228]:
fSample = pFolder + 'bookings.sample.csv.bz2'

In [229]:
df_sample = pd.read_csv(fSample,delimiter='^')
df_sample.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
13579,2013-01-07 00:00:00,1G,KW,303c3d997fdad556f25848cee2a32f5f,9730677f86eb9b2121c335c66bccff8d,6220263dff5279de15b2d6c0188adcc3,2013-01-07 00:00:00,17202,0,KWI,...,LHRDOHKWI,BC,E,Y,2013-01-23 20:30:00,2013-01-24 09:12:57,2,2013,1,NULL
13899,2013-01-26 00:00:00,1G,SA,10d45ef9223321e3d6c2e617e8178b90,a5bee175142c0a11b28b0812c9cf9ab9,c69cdefdffe9c70fdf6796426f3213b3,2013-01-07 00:00:00,20927,0,JED,...,CGKDXBJED,OU,U,Y,2013-04-09 07:25:00,2013-04-09 18:02:51,1,2013,1,NULL
43444,2013-01-15 00:00:00,1A,IR,61f232bdb022a925e3b76440f462b92a,e6eafbaac934e20b19c636a7d980dac3,d6dbf565b319c94e1713cfd08acb13d6,2012-12-25 00:00:00,387041,0,IKA,...,IKAISTORD,EU,E,Y,2013-01-18 08:20:00,2013-01-18 16:49:46,-1,2013,1,NULL
97932,2013-01-08 00:00:00,1A,IN,9e586aade06c70687736accbe5e98737,e92059401689ecc80a4aef7f065e1abc,9218323efad894d04df0a605693f0376,2012-12-25 00:00:00,866,9553,ADL,...,ADLSINDEL,DB,B,Y,2013-02-04 12:40:00,2013-02-04 22:06:25,-1,2013,1,NULL
85062,2013-03-19 00:00:00,1A,SA,ae50bfd9217dd423ac28cf7f39149eeb,cf898f870fab68ce029c4e6d7672815b,6fba6024953342a1472c8814c077fc92,2013-03-17 00:00:00,47438,0,CAI,...,CAIJEDGIZ,DG,Y,Y,2013-05-21 22:55:00,2013-05-22 07:13:00,3,2013,3,NULL


In [230]:
df_sample.columns=df_sample.columns.str.replace('\s+', '')

In [231]:
df_sample_sol= df_sample[['arr_port','pax','year']]

In [232]:
df_sample_sol=df_sample_sol[df_sample_sol['year']==2013]
df_sample_sol= df_sample[['arr_port','pax']]
df_sample_sol['arr_port']= df_sample['arr_port'].str.strip()

/home/dsc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [233]:
df_sample_sol.groupby(['arr_port']).sum().sort_values(by='pax',ascending=False).head(10)

,pax
arr_port,
LHR,1006
MCO,861
JFK,795
LAX,761
BKK,747
LAS,732
SFO,705
ORD,686
CDG,676


Note that we have arr_port with negative sum of pax, it's only a sampel and we can lost the good paxes, but if we find this in the full file maybe we have a bad data.

Now we have the exercice 2 solve to the sample, our new target is open the full file.

In [234]:
delimiter='^'
chunksize=500000
usecols=['arr_port','pax','year']
dataBookings = pd.read_csv(fBookings, delimiter=delimiter, compression='bz2', iterator=True, chunksize=chunksize,usecols=usecols)

In [235]:
df_total=pd.DataFrame()
for chunk in dataBookings:
    df=chunk[chunk['year']==2013]
    df=df[['arr_port','pax']]
    df['arr_port']=df['arr_port'].str.strip().str.upper()
    df=df.groupby('arr_port').sum().sort_values(by='pax',ascending=False)
    df_total=df_total.append(df)

In [236]:
df_total.groupby('arr_port').sum().sort_values(by='pax',ascending=False).head(10)

,pax
arr_port,
LHR,88809.0
MCO,70930.0
LAX,70530.0
LAS,69630.0
JFK,66270.0
CDG,64490.0
BKK,59460.0
MIA,58150.0
SFO,58000.0


## 3. Exercise 3
Plot the monthly number of searches for flights arriving at Malaga, Madrid or Barcelona